# 🎯 FPT Stock Forecasting with Market Regime-based Ensemble

## Approach
1. **Split data TRƯỚC** để tránh data leakage
2. **HMM Regime Detection** với 60-day window (train trên train set)
3. **Ensemble Models** (Linear + DLinear + NLinear) cho từng regime
4. **Predict 100 ngày** tiếp theo

## Key Anti-Leakage Measures
- Scaler fit trên train only
- HMM fit trên train only
- Regime assignment dùng model đã train

## 1. Setup & Import

In [ ]:
import os
import random
import warnings
import math
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from hmmlearn import hmm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 2. Load Data & Split FIRST (Anti-Leakage)

In [ ]:
# Load data
df = pd.read_csv('data/FPT_train.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

print(f"Shape: {df.shape}")
print(f"Date range: {df['time'].min().date()} → {df['time'].max().date()}")
display(df.head())

In [ ]:
# ⚠️ SPLIT FIRST - Anti-leakage
train_ratio = 0.7
val_ratio = 0.15
# test_ratio = 0.15 (implicit)

n_total = len(df)
train_end = int(n_total * train_ratio)
val_end = int(n_total * (train_ratio + val_ratio))

df_train = df.iloc[:train_end].copy()
df_val = df.iloc[train_end:val_end].copy()
df_test = df.iloc[val_end:].copy()

print(f"Train: {len(df_train)} ({df_train['time'].min().date()} → {df_train['time'].max().date()})")
print(f"Val:   {len(df_val)} ({df_val['time'].min().date()} → {df_val['time'].max().date()})")
print(f"Test:  {len(df_test)} ({df_test['time'].min().date()} → {df_test['time'].max().date()})")

## 3. Feature Engineering & Scaling (Train Only)

In [ ]:
def add_features(data):
    """Add log transforms and technical features"""
    data = data.copy()
    data['close_log'] = np.log(data['close'])
    data['open_log'] = np.log(data['open'])
    data['high_log'] = np.log(data['high'])
    data['low_log'] = np.log(data['low'])
    data['volume_log'] = np.log(data['volume'] + 1)
    data['hl_spread'] = (data['high'] - data['low']) / data['close']
    data['oc_spread'] = (data['close'] - data['open']) / data['open']
    return data

# Apply to all splits
df_train = add_features(df_train)
df_val = add_features(df_val)
df_test = add_features(df_test)

# Also keep full df for later
df_full = add_features(df)

print("Features added!")

In [ ]:
# ⚠️ FIT SCALER ON TRAIN ONLY
uni_scaler = StandardScaler()
uni_scaler.fit(df_train['close_log'].values.reshape(-1, 1))

# Transform all
df_train['close_scaled'] = uni_scaler.transform(df_train['close_log'].values.reshape(-1, 1)).flatten()
df_val['close_scaled'] = uni_scaler.transform(df_val['close_log'].values.reshape(-1, 1)).flatten()
df_test['close_scaled'] = uni_scaler.transform(df_test['close_log'].values.reshape(-1, 1)).flatten()
df_full['close_scaled'] = uni_scaler.transform(df_full['close_log'].values.reshape(-1, 1)).flatten()

print(f"Scaler mean: {uni_scaler.mean_[0]:.4f}, std: {uni_scaler.scale_[0]:.4f}")

## 4. HMM Regime Detection (Fit on Train Only)

In [ ]:
def compute_regime_features(data, window=60):
    """Compute features for regime detection"""
    data = data.copy()
    
    # Return in window days
    data['return'] = data['close'].pct_change(window) * 100
    
    # Volatility (annualized)
    data['daily_ret'] = data['close'].pct_change()
    data['volatility'] = data['daily_ret'].rolling(window).std() * np.sqrt(252) * 100
    
    # Trend
    def calc_trend(series):
        x = np.arange(len(series))
        slope = np.polyfit(x, series, 1)[0]
        return slope / series.mean() * 100
    
    data['trend'] = data['close'].rolling(window).apply(calc_trend, raw=False)
    
    return data

# Compute regime features
REGIME_WINDOW = 60
df_train_regime = compute_regime_features(df_train, REGIME_WINDOW)
df_full_regime = compute_regime_features(df_full, REGIME_WINDOW)

# Drop NaN for HMM training
df_train_regime_clean = df_train_regime.dropna().copy()
df_full_regime_clean = df_full_regime.dropna().copy()

print(f"Train regime samples: {len(df_train_regime_clean)}")
print(f"Full regime samples: {len(df_full_regime_clean)}")

In [ ]:
# ⚠️ FIT HMM ON TRAIN ONLY
regime_features = ['return', 'volatility', 'trend']

# Prepare train data
X_train_regime = df_train_regime_clean[regime_features].values

# Scale regime features (fit on train)
regime_scaler = StandardScaler()
X_train_regime_scaled = regime_scaler.fit_transform(X_train_regime)

# Fit HMM
N_REGIMES = 4
hmm_model = hmm.GaussianHMM(
    n_components=N_REGIMES,
    covariance_type="full",
    n_iter=1000,
    random_state=42
)
hmm_model.fit(X_train_regime_scaled)

print(f"HMM trained with {N_REGIMES} regimes!")

In [ ]:
# Predict regimes for ALL data (using trained HMM)
X_full_regime = df_full_regime_clean[regime_features].values
X_full_regime_scaled = regime_scaler.transform(X_full_regime)

# Predict
regimes_full = hmm_model.predict(X_full_regime_scaled)
df_full_regime_clean['regime'] = regimes_full

# Analyze regimes
regime_stats = df_full_regime_clean.groupby('regime').agg({
    'return': 'mean',
    'volatility': 'mean'
})

# Sort by return and assign names
sorted_regimes = regime_stats['return'].sort_values(ascending=False).index.tolist()
regime_names = {sorted_regimes[0]: 'Rally', sorted_regimes[1]: 'Uptrend', 
                sorted_regimes[2]: 'Sideway', sorted_regimes[3]: 'Downtrend'}
regime_colors = {sorted_regimes[0]: '#1B5E20', sorted_regimes[1]: '#4CAF50',
                 sorted_regimes[2]: '#9E9E9E', sorted_regimes[3]: '#C62828'}

df_full_regime_clean['regime_name'] = df_full_regime_clean['regime'].map(regime_names)

print("\nRegime Analysis:")
for regime in sorted_regimes:
    count = (df_full_regime_clean['regime'] == regime).sum()
    pct = count / len(df_full_regime_clean) * 100
    ret = regime_stats.loc[regime, 'return']
    vol = regime_stats.loc[regime, 'volatility']
    print(f"  {regime_names[regime]}: {pct:.1f}% | Return: {ret:+.1f}% | Vol: {vol:.0f}%")

In [ ]:
# Visualize regimes
fig, ax = plt.subplots(figsize=(16, 6))

for regime in sorted_regimes:
    mask = df_full_regime_clean['regime'] == regime
    ax.scatter(df_full_regime_clean.loc[mask, 'time'],
               df_full_regime_clean.loc[mask, 'close'],
               c=regime_colors[regime], label=regime_names[regime],
               alpha=0.6, s=15)

ax.set_title('FPT Stock with Market Regimes (HMM 60-day)', fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price (VND)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Model Definitions

In [ ]:
# Base Models
class Linear(nn.Module):
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.fc = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        return self.fc(x)


class NLinear(nn.Module):
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.fc = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        last = x[:, -1:]
        return self.fc(x - last) + last


class DLinear(nn.Module):
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.seq_len = seq_len
        self.kernel_size = max(3, seq_len // 4)
        self.fc_trend = nn.Linear(seq_len, pred_len)
        self.fc_seasonal = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        trend = x.unfold(-1, self.kernel_size, 1).mean(-1)
        pad_left = (self.seq_len - trend.size(-1)) // 2
        pad_right = self.seq_len - trend.size(-1) - pad_left
        trend = F.pad(trend, (pad_left, pad_right), mode='replicate')
        seasonal = x - trend
        return self.fc_trend(trend) + self.fc_seasonal(seasonal)

In [ ]:
# Regime-specific Ensemble
class RegimeEnsemble(nn.Module):
    """Ensemble of Linear + DLinear + NLinear with learnable weights"""
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.linear = Linear(seq_len, pred_len)
        self.dlinear = DLinear(seq_len, pred_len)
        self.nlinear = NLinear(seq_len, pred_len)
        self.weights = nn.Parameter(torch.ones(3) / 3)
        
    def forward(self, x):
        p1 = self.linear(x)
        p2 = self.dlinear(x)
        p3 = self.nlinear(x)
        w = F.softmax(self.weights, dim=0)
        return w[0] * p1 + w[1] * p2 + w[2] * p3
    
    def get_weights(self):
        return F.softmax(self.weights, dim=0).detach().cpu().numpy()

## 6. Dataset & Training Functions

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, series, regimes, seq_len, pred_len, target_regime=None):
        """
        Args:
            series: price data (scaled)
            regimes: regime labels for each timestep
            target_regime: if specified, only use samples from this regime
        """
        self.series = series.astype(np.float32)
        self.regimes = regimes
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.target_regime = target_regime
        
        # Find valid indices
        self.valid_indices = []
        for i in range(len(self.series) - seq_len - pred_len + 1):
            if target_regime is None:
                self.valid_indices.append(i)
            else:
                # Check if the END of input sequence is in target regime
                if i + seq_len - 1 < len(regimes) and regimes[i + seq_len - 1] == target_regime:
                    self.valid_indices.append(i)
    
    def __len__(self):
        return len(self.valid_indices)
    
    def __getitem__(self, idx):
        i = self.valid_indices[idx]
        x = self.series[i: i + self.seq_len]
        y = self.series[i + self.seq_len: i + self.seq_len + self.pred_len]
        return torch.from_numpy(x), torch.from_numpy(y)

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs, lr, patience, device):
    """Train model with early stopping"""
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    best_val_loss = float('inf')
    best_state = None
    counter = 0
    
    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validate
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                pred = model(x)
                val_loss += criterion(pred, y).item()
        
        val_loss /= len(val_loader)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = deepcopy(model.state_dict())
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                break
    
    model.load_state_dict(best_state)
    return model

In [ ]:
def evaluate_model(model, test_loader, device):
    """Evaluate model on test set"""
    model.eval()
    preds, targets = [], []
    
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            pred = model(x).cpu().numpy()
            preds.append(pred)
            targets.append(y.numpy())
    
    preds = np.concatenate(preds, axis=0)
    targets = np.concatenate(targets, axis=0)
    
    rmse = np.sqrt(mean_squared_error(targets.flatten(), preds.flatten()))
    mae = mean_absolute_error(targets.flatten(), preds.flatten())
    
    return rmse, mae, preds, targets

## 7. Train Ensemble Model for Each Regime

In [ ]:
# Config
SEQ_LEN = 60  # Match regime window
PRED_LEN = 100
BATCH_SIZE = 32
NUM_EPOCHS = 300
PATIENCE = 50
LR = 1e-3

# Prepare data with regimes
# Merge regimes back to full dataframe
series_full = df_full_regime_clean['close_scaled'].values
regimes_full_arr = df_full_regime_clean['regime'].values
times_full = df_full_regime_clean['time'].values

# Get train/val/test indices based on time
train_end_time = df_train['time'].max()
val_end_time = df_val['time'].max()

train_mask = df_full_regime_clean['time'] <= train_end_time
val_mask = (df_full_regime_clean['time'] > train_end_time) & (df_full_regime_clean['time'] <= val_end_time)
test_mask = df_full_regime_clean['time'] > val_end_time

print(f"Train samples (with regime): {train_mask.sum()}")
print(f"Val samples: {val_mask.sum()}")
print(f"Test samples: {test_mask.sum()}")

In [ ]:
# Train ensemble for each regime
regime_models = {}
regime_results = []

for regime in sorted_regimes:
    regime_name = regime_names[regime]
    print(f"\n{'='*60}")
    print(f"Training Ensemble for Regime: {regime_name}")
    print(f"{'='*60}")
    
    # Create dataset for this regime
    dataset = TimeSeriesDataset(
        series_full, regimes_full_arr, SEQ_LEN, PRED_LEN, target_regime=regime
    )
    
    if len(dataset) < 10:
        print(f"  ⚠️ Not enough samples ({len(dataset)}), skipping...")
        continue
    
    # Split into train/val
    total = len(dataset)
    train_len = int(total * 0.8)
    
    train_subset = torch.utils.data.Subset(dataset, list(range(train_len)))
    val_subset = torch.utils.data.Subset(dataset, list(range(train_len, total)))
    
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE)
    
    print(f"  Samples: {len(dataset)} (train: {train_len}, val: {total - train_len})")
    
    # Create and train model
    model = RegimeEnsemble(SEQ_LEN, PRED_LEN)
    model = train_model(model, train_loader, val_loader, NUM_EPOCHS, LR, PATIENCE, device)
    
    # Get ensemble weights
    weights = model.get_weights()
    print(f"  Weights: Linear={weights[0]:.3f}, DLinear={weights[1]:.3f}, NLinear={weights[2]:.3f}")
    
    # Evaluate
    rmse, mae, _, _ = evaluate_model(model, val_loader, device)
    print(f"  Val RMSE: {rmse:.4f}, MAE: {mae:.4f}")
    
    regime_models[regime] = model
    regime_results.append({
        'Regime': regime_name,
        'Samples': len(dataset),
        'RMSE': rmse,
        'MAE': mae,
        'Linear_W': weights[0],
        'DLinear_W': weights[1],
        'NLinear_W': weights[2]
    })

print("\n" + "="*60)
print("Training Complete!")
print("="*60)

In [ ]:
# Display results
results_df = pd.DataFrame(regime_results)
print("\nRegime Model Results:")
display(results_df)

## 8. Train Global Ensemble (All Data) for Comparison

In [ ]:
# Global model (no regime filtering)
print("\nTraining Global Ensemble (all regimes combined)...")

global_dataset = TimeSeriesDataset(
    series_full, regimes_full_arr, SEQ_LEN, PRED_LEN, target_regime=None
)

total = len(global_dataset)
train_len = int(total * 0.8)

train_subset = torch.utils.data.Subset(global_dataset, list(range(train_len)))
val_subset = torch.utils.data.Subset(global_dataset, list(range(train_len, total)))

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE)

global_model = RegimeEnsemble(SEQ_LEN, PRED_LEN)
global_model = train_model(global_model, train_loader, val_loader, NUM_EPOCHS, LR, PATIENCE, device)

global_weights = global_model.get_weights()
global_rmse, global_mae, _, _ = evaluate_model(global_model, val_loader, device)

print(f"Global Ensemble:")
print(f"  Samples: {len(global_dataset)}")
print(f"  Weights: Linear={global_weights[0]:.3f}, DLinear={global_weights[1]:.3f}, NLinear={global_weights[2]:.3f}")
print(f"  Val RMSE: {global_rmse:.4f}, MAE: {global_mae:.4f}")

## 9. Retrain Best Models on Full Data & Generate Forecast

In [ ]:
# Retrain all regime models on 90% of their data
print("\nRetraining regime models on 90% data...")

final_regime_models = {}

for regime in sorted_regimes:
    if regime not in regime_models:
        continue
    
    regime_name = regime_names[regime]
    
    dataset = TimeSeriesDataset(
        series_full, regimes_full_arr, SEQ_LEN, PRED_LEN, target_regime=regime
    )
    
    if len(dataset) < 10:
        continue
    
    total = len(dataset)
    train_len = int(total * 0.9)
    
    train_subset = torch.utils.data.Subset(dataset, list(range(train_len)))
    val_subset = torch.utils.data.Subset(dataset, list(range(train_len, total)))
    
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE)
    
    model = RegimeEnsemble(SEQ_LEN, PRED_LEN)
    model = train_model(model, train_loader, val_loader, NUM_EPOCHS, LR, PATIENCE, device)
    
    final_regime_models[regime] = model
    print(f"  {regime_name}: ✓")

# Also retrain global model
total = len(global_dataset)
train_len = int(total * 0.9)
train_subset = torch.utils.data.Subset(global_dataset, list(range(train_len)))
val_subset = torch.utils.data.Subset(global_dataset, list(range(train_len, total)))
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE)

final_global_model = RegimeEnsemble(SEQ_LEN, PRED_LEN)
final_global_model = train_model(final_global_model, train_loader, val_loader, NUM_EPOCHS, LR, PATIENCE, device)
print("  Global: ✓")

In [ ]:
def forecast_future(model, input_data, scaler, device):
    """Generate forecast for future"""
    model.eval()
    
    with torch.no_grad():
        x = torch.from_numpy(input_data.astype(np.float32)).unsqueeze(0).to(device)
        pred_scaled = model(x).cpu().numpy().flatten()
    
    # Inverse transform
    pred_log = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
    pred_price = np.exp(pred_log)
    
    return pred_price

In [ ]:
# Determine current regime
current_regime = regimes_full_arr[-1]
current_regime_name = regime_names[current_regime]
print(f"\nCurrent market regime: {current_regime_name}")

# Get last SEQ_LEN data
input_data = series_full[-SEQ_LEN:]

# Generate forecasts from both regime-specific and global models
print("\nGenerating 100-day forecasts...")

# Regime-specific forecast
if current_regime in final_regime_models:
    regime_forecast = forecast_future(final_regime_models[current_regime], input_data, uni_scaler, device)
    print(f"  Regime ({current_regime_name}) forecast: {regime_forecast[0]:.2f} → {regime_forecast[-1]:.2f}")
else:
    regime_forecast = None
    print(f"  ⚠️ No model for regime {current_regime_name}")

# Global forecast
global_forecast = forecast_future(final_global_model, input_data, uni_scaler, device)
print(f"  Global forecast: {global_forecast[0]:.2f} → {global_forecast[-1]:.2f}")

In [ ]:
# Choose best forecast (regime-specific if available)
if regime_forecast is not None:
    final_forecast = regime_forecast
    forecast_type = f"Regime-{current_regime_name}"
else:
    final_forecast = global_forecast
    forecast_type = "Global"

print(f"\nUsing {forecast_type} forecast")
print(f"Forecast range: {final_forecast.min():.2f} - {final_forecast.max():.2f} VND")

## 10. Create Submission

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame({
    'id': np.arange(1, PRED_LEN + 1),
    'close': final_forecast
})

# Save
os.makedirs('submissions', exist_ok=True)
submission_path = f'submissions/submission_regime_ensemble_{current_regime_name.lower()}.csv'
submission.to_csv(submission_path, index=False)

print(f"\n✅ Submission saved: {submission_path}")
print(f"Rows: {len(submission)}")
print("\nPreview:")
display(submission.head(10))
display(submission.tail(5))

In [ ]:
# Also save global forecast for comparison
submission_global = pd.DataFrame({
    'id': np.arange(1, PRED_LEN + 1),
    'close': global_forecast
})
submission_global.to_csv('submissions/submission_global_ensemble.csv', index=False)
print("Also saved: submissions/submission_global_ensemble.csv")

## 11. Visualization

In [ ]:
# Plot historical + forecast
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Plot 1: Full history with regimes
ax = axes[0]
for regime in sorted_regimes:
    mask = df_full_regime_clean['regime'] == regime
    ax.scatter(df_full_regime_clean.loc[mask, 'time'],
               df_full_regime_clean.loc[mask, 'close'],
               c=regime_colors[regime], label=regime_names[regime],
               alpha=0.5, s=10)

ax.set_title('FPT Stock with Market Regimes', fontweight='bold')
ax.set_ylabel('Price (VND)')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# Plot 2: Recent history + forecast
ax = axes[1]

hist_days = 150
hist_dates = df_full_regime_clean['time'].iloc[-hist_days:]
hist_prices = df_full_regime_clean['close'].iloc[-hist_days:]

# Future dates
last_date = df_full_regime_clean['time'].iloc[-1]
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=PRED_LEN, freq='B')

ax.plot(hist_dates, hist_prices, 'b-', linewidth=2, label='Historical')
ax.plot(future_dates, final_forecast, 'r--', linewidth=2, label=f'Forecast ({forecast_type})')

if regime_forecast is not None and not np.array_equal(regime_forecast, global_forecast):
    ax.plot(future_dates, global_forecast, 'g:', linewidth=1.5, alpha=0.7, label='Global Ensemble')

ax.axvline(last_date, color='gray', linestyle=':', alpha=0.7, label='Forecast start')

ax.set_title(f'100-Day Forecast (Current Regime: {current_regime_name})', fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price (VND)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('submissions/forecast_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Summary
print("\n" + "="*70)
print("📊 SUMMARY")
print("="*70)
print(f"\nData: FPT Stock ({df['time'].min().date()} → {df['time'].max().date()})")
print(f"Regime Window: {REGIME_WINDOW} days")
print(f"Sequence Length: {SEQ_LEN} days")
print(f"Prediction Length: {PRED_LEN} days")
print(f"\nCurrent Regime: {current_regime_name}")
print(f"Forecast Type: {forecast_type}")
print(f"\nForecast Statistics:")
print(f"  Start: {final_forecast[0]:.2f} VND")
print(f"  End: {final_forecast[-1]:.2f} VND")
print(f"  Min: {final_forecast.min():.2f} VND")
print(f"  Max: {final_forecast.max():.2f} VND")
print(f"  Mean: {final_forecast.mean():.2f} VND")
print(f"\nSubmission: {submission_path}")
print("="*70)